In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import gen_css
import add_current
import BaiduTrans

In [2]:
aer = requests.get('https://www.aeaweb.org/journals/aer')

In [3]:
aer_data = BeautifulSoup(aer.text, 'html.parser')

In [4]:
current_link = aer_data.find(class_="current-issue-link")
current_link

<a class="current-issue-link button highlight" href="/issues/764">Current Issue</a>

In [5]:
current_url = 'https://www.aeaweb.org' + current_link.get('href')
current_url

'https://www.aeaweb.org/issues/764'

In [6]:
issue = re.search('iss.+',current_url)[0]
issue = re.sub(r"/", "_", issue)
issue

'issues_764'

In [7]:
current_data = requests.get(current_url)

In [8]:
data = BeautifulSoup(current_data.text, 'html.parser')

In [9]:
articles = data.find_all(class_='journal-article')

In [10]:
article_link = 'https://www.aeaweb.org' + articles[3].find('a').get('href')
article_link

'https://www.aeaweb.org/articles?id=10.1257/aer.20220425'

In [11]:
article_requests = requests.get(article_link)

In [12]:
article_data = BeautifulSoup(article_requests.text, 'html.parser')

In [13]:
article_data.find(class_='title').text

'Local Economic and Political Effects of Trade Deals: Evidence from NAFTA'

In [14]:
ori_name = article_data.find(class_='attribution').text
ori_name

'\n\n         Jiwon Choi    \n\n         Ilyana Kuziemko    \n\n         Ebonya Washington    \n\n         Gavin Wright    \n'

In [15]:
names = re.findall(r'\b[A-Za-z]+(?:\s[A-Za-z]+)+\b', ori_name)
name = ', '.join(names)
name

'Jiwon Choi, Ilyana Kuziemko, Ebonya Washington, Gavin Wright'

In [16]:
ori_abstract = article_data.find(class_='abstract').text
abstract = cleaned_text = re.sub(r"Abstract|\t|\n", "", ori_abstract)
abstract

'Why have white, less-educated voters left the Democratic Party? We highlight the role of the 1994 North American Free Trade Agreement (NAFTA). In event-study analysis, we demonstrate that counties whose 1990 employment depended on industries vulnerable to NAFTA suffered large and persistent employment losses after its implementation. Voters in these counties (and protectionist voters regardless of geography) turned away from the party of President Clinton, who promoted the agreement. This shift is larger for whites (especially men and those without a college degree) and social conservatives, suggesting that racial identity and social-issue positions mediate reactions to economic policies.'

In [17]:
title = []
name = []
abstracts = []
for art_link in articles:
    try:
        art_link = 'https://www.aeaweb.org' + art_link.find('a').get('href')
        article_requests = requests.get(art_link)
        article_data = BeautifulSoup(article_requests.text, 'html.parser')
        title.append(article_data.find(class_='title').text)
        ori_name = article_data.find(class_='attribution').text
        re_name = re.findall(r'\b[A-Za-z]+(?:\s[A-Za-z]+)+\b', ori_name)
        names = ', '.join(re_name)
        name.append(names)
        ori_abstract = article_data.find(class_='abstract').text
        abstract = cleaned_text = re.sub(r"Abstract|\t|\n", "", ori_abstract)
        abstracts.append(abstract)
    except:
        pass
article = [*zip(title, name, abstracts)]
article

[('Front Matter',
  '',
  'Why have white, less-educated voters left the Democratic Party? We highlight the role of the 1994 North American Free Trade Agreement (NAFTA). In event-study analysis, we demonstrate that counties whose 1990 employment depended on industries vulnerable to NAFTA suffered large and persistent employment losses after its implementation. Voters in these counties (and protectionist voters regardless of geography) turned away from the party of President Clinton, who promoted the agreement. This shift is larger for whites (especially men and those without a college degree) and social conservatives, suggesting that racial identity and social-issue positions mediate reactions to economic policies.'),
 ('Nobel Lecture: An Evolving Economic Force',
  'Claudia Goldin',
  "South Koreans appear to be preoccupied with their offspring's education and also have the lowest total fertility rate in the world. We propose a novel theory with status externalities and endogenous fer

In [18]:
with open('aer/AER_' + issue + '.html','w',encoding='UTF-8') as f:
    f.write(gen_css.gen_content_from_list(title))
    f.write(gen_css.gen_abstract_list(list(article)))

In [19]:
add_current.add('AER_' + issue, time.ctime(), 'aer/AER_' + issue + '.html')

In [21]:
abstract_cn = BaiduTrans.trans_list(abstracts)
title_cn = BaiduTrans.trans_list(title)
article_cn = [*zip(title_cn, names, abstract_cn)]
with open('jpe/JPE_CN_' + issue + '.html','w',encoding='UTF-8') as f:
    f.write(gen_css.gen_content_from_list(title_cn))
    f.write(gen_css.gen_abstract_list(list(article_cn)))
add_current.add('JPE_CN_' + issue, time.ctime(), 'jpe/JPE_CN_' + issue + '.html')

KeyboardInterrupt: 

In [24]:
title_cn = BaiduTrans.trans_list(title)
title_cn = title_cn[1:]

KeyboardInterrupt: 

In [23]:
abstract_cn = BaiduTrans.trans_list(abstracts)
abstract_cn

['为什么受教育程度较低的白人选民离开了民主党？我们强调1994年《北美自由贸易协定》的作用。在事件研究分析中，我们证明，1990年就业依赖于易受北美自由贸易协定影响的行业的县在实施后遭受了巨大而持续的就业损失。这些县的选民（以及不分地理位置的保护主义选民）拒绝了推动该协议的克林顿总统的政党。这种转变对白人（尤其是男性和没有大学学历的人）和社会保守派来说更大，这表明种族认同和社会问题立场会调节对经济政策的反应。',
 '韩国人似乎专注于子女的教育，而且总生育率也是世界上最低的。我们提出了一个新的理论，将这些事实与地位外部性和内生生育率联系起来，其动机是关于私立教育溢出的新的经验证据。使用一个针对韩国的定量模型，我们发现如果没有外部性，生育率将高出28%。我们探讨了政府政策的影响：产前转移或教育税可以提高生育率并减少教育支出。22%的教育税加上适度的产前转移，最大限度地提高了这一代人的福利。',
 '我们研究了学习偏差的稳健福利比较（指定错误的贝叶斯和某些形式的非贝叶斯更新）。给定真实的信号分布，如果在所有决策问题中产生较低的客观预期收益，我们认为一种偏差比另一种更有害。我们在静态和动态设置中描述这种排名。当静态表征逐个信号比较后验信号时，动态表征采用了一个“效率指数”来衡量信念收敛的速度。我们量化并比较了几个有充分记录的偏差的严重程度。我们还强调了静态和动态排名之间的差异，以及一些“大”的偏见在动态上胜过其他“微不足道”的偏见。',
 '我们研究了一个国际货币体系的商业周期模型，其特征是对安全美元债券的需求随时间变化，美国的风险承受能力高于世界其他地区，以及名义刚性。向安全地带的逃亡导致美元升值，全球产出下降。因此，美元债券的风险溢价为负，美国持有以美元为单位的杠杆资本金融投资组合。我们量化了安全冲击和风险承受能力异质性对全球宏观经济波动、美国外部调整和政策传导的影响，如美元互换额度。',
 '本文确定了增加产后工作激励对母亲长期职业生涯的影响。我们根据初产的时间和1993年扩大劳动所得税抵免的资格，利用了母亲之间工作激励的差异。第一次分娩后10至19年，出生后立即（“早期”）而不是3-6年后（“晚期”）暴露于扩张的单身母亲，有0.62年的工作经验，以工作为条件的收入高4.2%。我们发现，收入的增加主要是由于工作经验的增加而导致工资的提高。',
 '我们开发了一个基

In [25]:
title_cn = title_cn[1:]

In [26]:
title_cn

['诺贝尔演讲：一股不断发展的经济力量',
 '贸易协议对地方经济和政治的影响：来自北美自由贸易协定的证据',
 '韩国教育的地位外部性与低出生率',
 '有偏见学习的福利比较',
 '安全飞行与国际风险分担',
 '生育后工作激励的长期效应',
 '基于任务的歧视',
 '信息技术与规模回报',
 '打破性别障碍：对从事粉领工作的男性的实验证据',
 '金融准入和劳动力市场结果：来自信用彩票的证据']

In [29]:
article_cn = [*zip(title_cn, name, abstract_cn)]
with open('jpe/JPE_CN_' + issue + '.html','w',encoding='UTF-8') as f:
    f.write(gen_css.gen_content_from_list(title_cn))
    f.write(gen_css.gen_abstract_list(list(article_cn)))
add_current.add('JPE_CN_' + issue, time.ctime(), 'jpe/JPE_CN_' + issue + '.html')